# MNIST with Summaries

original: https://github.com/normanheckscher/mnist-tensorboard-embeddings/blob/master/mnist_with_summaries.py  
( also refer-to: https://qiita.com/FukuharaYohei/items/70f702ee85e23b5b1e26 )

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

In [ ]:
from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist    import input_data

## Constants

In [ ]:
TO_EMBED_COUNT = 1000
CUR_DIR = os.path.abspath(os.path.curdir)
data_dir = "MNIST_data"
log_dir = "mnist_sample"
sprite_image_filename = "mnist_sprite.png"

max_steps = 1000
learning_rate = 0.001

In [ ]:
metadata_csv_path = os.path.join(CUR_DIR, log_dir, 'projector', 'metadata.tsv')
sprite_image_path = os.path.join(CUR_DIR, log_dir, sprite_image_filename)

## Data preparations

In [ ]:
# Import data
mnist = input_data.read_data_sets(data_dir, one_hot=True)

In [ ]:
def save_metadata(file, mnist=mnist):
    tf.gfile.MakeDirs(os.path.dirname(file))
    with open(file, 'w') as f:
        # f.write('id\tchar\n')
        for i in range(TO_EMBED_COUNT):
            c = np.nonzero(mnist.test.labels)[1][i]
            f.write('{}\n'.format(c))

In [ ]:
save_metadata(metadata_csv_path)

In [ ]:
def create_sprite_image(images):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    # MNISTは28ピクセル四方 
    img_h = images.shape[1]
    img_w = images.shape[2]

    # 画像数の平方根(切上)を計算(Sprite Imageの1辺の長さに使用)
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))

    # 全要素0の配列作成
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))

    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j

            # 画像がある限り実行(n_plotsが割り切れないためSprite Imageは少し余る)
            if this_filter < images.shape[0]:

                # Sprite Imageの所定の配列に画像を挿入
                spriteimage[i * img_h:(i + 1) * img_h, j * img_w:(j + 1) * img_w] = images[this_filter]

    return spriteimage

In [ ]:
# Sprite Image生成
sprite_image = create_sprite_image(1. - np.reshape(mnist.test.images[:TO_EMBED_COUNT], (-1, 28, 28)))

In [ ]:
# Sprite Image 確認
plt.imshow(sprite_image, cmap='gray')

In [ ]:
# Sprite Image保存
plt.imsave(sprite_image_path, sprite_image,cmap='gray')

## Graph

In [ ]:
# Input placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

In [ ]:
## Summary Image
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

In [ ]:
# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [ ]:
def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [ ]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations

In [ ]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

In [ ]:
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

In [ ]:
# Do not apply softmax activation yet, see below.
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

In [ ]:
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)

In [ ]:
tf.summary.scalar('cross_entropy', cross_entropy)

In [ ]:
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

## for Embeddings (Projector)

In [ ]:
# Input set for Embedded TensorBoard visualization
# Performed with cpu to conserve memory and processing power
with tf.device("/cpu:0"):
    embedding1 = tf.Variable(tf.stack(mnist.test.images[:TO_EMBED_COUNT], axis=0), trainable=False, name='embedding1')
    embedding2 = tf.Variable(np.zeros((TO_EMBED_COUNT, 500)), trainable=False, dtype=tf.float32, name='embedding2')
    assign_embedding2 = tf.assign(embedding2, hidden1)
    embedding3 = tf.Variable(np.zeros((TO_EMBED_COUNT, 10)), trainable=False, dtype=tf.float32, name='embedding3')
    assign_embedding3 = tf.assign(embedding3, y)

## Initializations

In [ ]:
initialize_op = tf.global_variables_initializer()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(log_dir, 'train'), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(log_dir, 'test'))

In [ ]:
sess.run(initialize_op)

In [ ]:
saver = tf.train.Saver()

In [ ]:
# Projector 用 SummaryWriter
writer = tf.summary.FileWriter(os.path.join(CUR_DIR, log_dir, 'projector'), sess.graph)

In [ ]:
# Add embedding tensorboard visualization.
config = projector.ProjectorConfig()
for embedding_var in (embedding1, embedding2, embedding3):
    # embedding_i
    embed = config.embeddings.add()
    embed.tensor_name = embedding_var.name
    embed.metadata_path = metadata_csv_path
    embed.sprite.image_path = sprite_image_path
    embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(writer, config)

## Trainings

Train the model, and also write summaries.  
Every 10th step, measure test-set accuracy, and write test summaries  
All other steps, run train_step on training data, & add training summaries

In [ ]:
def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train:
        xs, ys = mnist.train.next_batch(100)
        k = 0.5
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

In [ ]:
for i in range(max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))

    else:  # Record train set summaries, and train
        if i % 100 == 99:  # Record execution stats
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step],
                                  feed_dict=feed_dict(True),
                                  options=run_options,
                                  run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)

            print('Adding run metadata for', i)

        else:  # Record a summary
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            train_writer.add_summary(summary, i)

## Embeddings

In [ ]:
sess.run((assign_embedding2, assign_embedding3), 
         feed_dict={x: mnist.test.images[:TO_EMBED_COUNT],  keep_prob: 1.0})
saver.save(sess, os.path.join(log_dir, 'projector', 'a_model.ckpt'), global_step=max_steps)

## Termination

In [ ]:
train_writer.close()
test_writer.close()
writer.close()

run: `tensorboard --logdir=$(pwd)/mnist_sample`  
( or run: `./run_tensorboard_mnist_demo.sh` )